In [8]:
#Import & Select Data Process
import csv
import pandas as pd
#import file for evaluation process
from tensorflow.python.keras import Input
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import median_absolute_error, mean_absolute_percentage_error
from numpy import arange

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500


In [ ]:
#import data
data = pd.read_csv("train.csv")

#according to the 
#in land sale thing they mostly consider :
#TotalLandSize, Building Condition, Environment Condition and Sale Condition
#so the thing that related to the above detail will be add to data frame 
# and use the number only about the text will be list below here
#Street, Alley, Sale Condition, GrLive Area, Electrical,MsZoning, Sale Type
df = data.drop("Id", axis=1)
# list(set(df.dtypes.tolist()))
# df.info()
#select only integer and float variable
df_num = df.select_dtypes(include=["float64","int64"])
df_num.head(3)

In [ ]:
#Cleaning Process
#the reason there is no split column is the data is already pretty in good
#Array shape
#
# for a in data.index:
#  raw = str(data.loc[a, "ScreenResolution"])
#  if (' ' in raw) == False:
#   data.loc[a, "ScreenResolution"] = " " + raw
#
#-------------

In [ ]:
# Split dataset into train and validation:
train_1, validation_1 = split_dataset(df, computer_frac=0.8)
plt.scatter(train_1['Inches'].astype(int), train_1['Price'].astype(int), c='blue', alpha=0.3)
plt.scatter(validation_1['Inches'].astype(int), validation_1['Price'].astype(int), c='red', alpha=0.3)
plt.xlabel('Inches')
plt.ylabel('Price')
plt.legend(['train set', 'validation set'], framealpha=0.3)
plt.show()

In [ ]:
#visualize df_num 
df_num.hist(figsize=(16,20),bins=50, xlabelsize=8, ylabelsie=8)

In [ ]:
# Create model:
model = Sequential()
model.add(Input(shape=(1,)))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(1, activation='linear'))
print(model.summary())

In [ ]:
#Evaluation Process

# Train:
loss = 'mse'
metric = 'mae'
#accurate epoch is 1750 but due to low CPU use low Epochs
epochs = 500

x_train, y_train = train['x'], train['y']
x_val, y_val = validation['x'], validation['y']
model.compile(loss=loss, optimizer='adam', metrics=[metric])
history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, verbose=1, validation_data=(x_val, y_val))
#